<img align='left' src = https://linea.org.br/wp-content/themes/LIneA/imagens/logo-header.jpg width=150 style='padding: 20px'> 

## Tutorial: particionamento de dados com Hipscat  

Passo-a-passo para conversão de catálogos astronômicos para o formato Hipscat. 

Contato: Julia Gschwend ([julia@linea.org.br](mailto:julia@linea.org.br))
<br>
Última verificação: 15/01/2024
<br>



#### Acknowledgements

<font color='red'> texto pedindo citação - ver modelo no jupyterhub do dirac/lincc </font>

'_Esta pesquisa utilizou recursos computacionais da Associação Laboratório Interinstitucional de e-Astronomia (LIneA) com o apoio financeiro do INCT do e-Universo (Processo n.º 465376/2014-2)._' 

LIneA

LINCC Frameworks 


### Introdução

<font color='red'> conceito healpix </font>

<font color='red'> conceito hipscat </font>

<font color='red'> conceito x-match espacial </font>

<font color='red'> desafios técnicos, escalabilidade, objetos nas bordas, etc </font>



### Caracterização da amostra de exemplo 

Amostra pública de resdhifts espectroscópico

82M     public_specz_compilation.pq 




In [ ]:
from pzserver import PzServer 
import matplotlib.pyplot as plt
%reload_ext autoreload 
%autoreload 2

In [ ]:
with open('/home/julia/hipscat_data/token.txt', 'r') as file:
    token = file.read().rstrip()

In [ ]:
pz_server = PzServer(token=token, host="pz-dev") 

In [ ]:
specz = pz_server.get_product('26_public_specz_compilation')

In [ ]:
specz.display_metadata()

In [ ]:
specz.data.describe()

In [ ]:
specz.plot()

<font color='red'> TBD: mapa de densidade de objetos com bokeh/datashader <font/>

Reescrever o arquivo original, com ObjectID fictício (inteiro sequencial)

In [ ]:
type(specz.data)

In [ ]:
df_reset = specz.data.reset_index()
df_reset

In [ ]:
df_reset.to_parquet('/home/julia/hipscat_data/specz.pq', index=False)

In [ ]:
df_reset.to_csv('/home/julia/hipscat_data/specz.csv', index=False)

### Instalação do `hipscat_import` 

```shell
pip install hipscat-import

```




### Execução - conversão dos arquivos para formato Hipscat

https://hipscat-import.readthedocs.io/en/latest/catalogs/public/allwise.html


``` 
wget -x http://irsa.ipac.caltech.edu/data/download/wise-allwise/README.txt
wget -x http://irsa.ipac.caltech.edu/data/download/wise-allwise/wise-allwise-cat.bz2.sizes.txt
wget -x http://irsa.ipac.caltech.edu/data/download/wise-allwise/wise-allwise-cat.bz2.md5.txt
wget -x http://irsa.ipac.caltech.edu/data/download/wise-allwise/wise-allwise-cat-part01.bz2
wget -x http://irsa.ipac.caltech.edu/data/download/wise-allwise/wise-allwise-cat-part02.bz2
wget -x http://irsa.ipac.caltech.edu/data/download/wise-allwise/wise-allwise-cat-part03.bz2 
``` 

In [ ]:
import pandas as pd

import hipscat_import.pipeline as runner
from hipscat_import.catalog.arguments import ImportArguments
from hipscat_import.catalog.file_readers import CsvReader

In [ ]:
# Load the column names and types from a side file.
type_frame = pd.read_csv("/home/julia/hipscat_data/pzserver/specz_types.csv")
type_map = dict(zip(type_frame["name"], type_frame["type"]))

In [ ]:
type_map

### Catalog Import Arguments

In [ ]:

# args = ImportArguments(
#     output_artifact_name="allwise",
#     input_path="/path/to/allwise/",
#     input_format="csv.bz2",
#     file_reader=CsvReader(
#         header=None,
#         separator="|",
#         column_names=type_frame["name"].values.tolist(),
#         type_map=type_map,
#         chunksize=250_000,
#     ),
#     use_schema_file="allwise_schema.parquet",
#     ra_column="ra",
#     dec_column="dec",
#     sort_columns="source_id",
#     pixel_threshold=1_000_000,
#     highest_healpix_order=7,
#     output_path="/path/to/catalogs/",
# )
# runner.pipeline(args)

In [ ]:
args = ImportArguments(
    sort_columns="index",
    ra_column="ra",
    dec_column="dec",
    input_path="/home/julia/hipscat_data/pzserver/input",
    input_format="csv",
    output_artifact_name="test_cat_specz",
    output_path="/home/julia/hipscat_data/pzserver/output",
)

In [ ]:
runner.pipeline(args)

### Análise dos resultados 

visualização dos dados em formato hipscat (ex: mapa de densidade de pixels)

### Instação do LSDB


### X-matching usando o LSDB

### Análise dos resultados do X-matching